In [6]:
import os

print("Notebook cwd:   ", os.getcwd())
print("One up (`..`):   ", os.listdir(".."))
print("Two up (`../..`):", os.listdir(os.path.join("..", "..")))

# If you see a 'data' folder in the two‑up listing, also check it:
if 'data' in os.listdir(os.path.join("..", "..")):
    print("\nContents of ../../data:", os.listdir(os.path.join("..", "..", "data")))


Notebook cwd:    d:\Yolo-intern-project\knife-detector-\backend\Research_notebooks
One up (`..`):    ['config.py', 'image_utils.py', 'main.py', 'models', 'Research_notebooks', 'routes', 'services', 'utils', '__init__.py', '__pycache__']
Two up (`../..`): ['.DS_Store', '.git', '.gitignore', '.idea', 'backend', 'data', 'Documentation.txt', 'README.md', 'requirements.txt', 'training', 'venv']

Contents of ../../data: ['labeled-dataset', 'labeled-dataset.zip', 'raw_images']


In [11]:
import zipfile
import os

zip_path   = os.path.join("..", "..", "data", "labeled-dataset.zip")
extract_to = os.path.join(os.getcwd())

os.makedirs(extract_to, exist_ok=True)

with zipfile.ZipFile(zip_path, "r") as z:
    z.extractall(extract_to)

print(f"Extracted {os.path.basename(zip_path)} → {extract_to}")


Extracted labeled-dataset.zip → d:\Yolo-intern-project\knife-detector-\backend\Research_notebooks


In [ ]:
import random
from pathlib import Path
import shutil
import os

BASE       = Path.cwd()                    
SRC_ROOT   = BASE / "labeled-dataset"       
TRAIN_FRAC = 0.8        

# source dirs
IMG_SRC  = SRC_ROOT / "images"
LBL_SRC  = SRC_ROOT / "labels"

# destination dirs
DST_ROOT = BASE / "data"
TRAIN_IMG = DST_ROOT / "train"      / "images"
TRAIN_LBL = DST_ROOT / "train"      / "labels"
VAL_IMG   = DST_ROOT / "validation" / "images"
VAL_LBL   = DST_ROOT / "validation" / "labels"

for d in (TRAIN_IMG, TRAIN_LBL, VAL_IMG, VAL_LBL):
    d.mkdir(parents=True, exist_ok=True)

# gather all images and then split
all_imgs = list(IMG_SRC.glob("*.*"))    
random.shuffle(all_imgs)
split_idx = int(len(all_imgs) * TRAIN_FRAC)

train_imgs = all_imgs[:split_idx]
val_imgs   = all_imgs[split_idx:]

print(f"Total images: {len(all_imgs)}   → train: {len(train_imgs)}, val: {len(val_imgs)}")

def copy_set(img_list, dst_img_dir, dst_lbl_dir):
    for img in img_list:
        lbl = LBL_SRC / (img.stem + ".txt")
        shutil.copy(img, dst_img_dir / img.name)
        if lbl.exists():
            shutil.copy(lbl, dst_lbl_dir / lbl.name)

# perform copies
copy_set(train_imgs, TRAIN_IMG, TRAIN_LBL)
copy_set(val_imgs,   VAL_IMG,   VAL_LBL)

print("Done splitting.")


Total images: 22   → train: 17, val: 5
Done splitting.


In [13]:
!pip install ultralytics


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import yaml
from pathlib import Path

BASE = Path.cwd()                        
CLASSES_TXT = BASE / "labeled-dataset" / "classes.txt"
DATA_YAML   = BASE / "data.yaml"

# 1) Read classes.txt
if not CLASSES_TXT.exists():
    raise FileNotFoundError(f"classes.txt not found at {CLASSES_TXT}")
classes = [l.strip() for l in CLASSES_TXT.read_text().splitlines() if l.strip()]
nc = len(classes)

# 2) Build data dict
data = {
    "path": str(BASE / "data"),        
    "train": "train/images",
    "val":   "validation/images",
    "nc":    nc,
    "names": classes
}

# 3) Write to YAML
DATA_YAML.write_text(yaml.dump(data, sort_keys=False))
print(f"✔︎ Created {DATA_YAML}\n")

# 4) Print out the file
print("----- data.yaml -----")
print(DATA_YAML.read_text())


✔︎ Created d:\Yolo-intern-project\knife-detector-\backend\Research_notebooks\data.yaml

----- data.yaml -----
path: d:\Yolo-intern-project\knife-detector-\backend\Research_notebooks\data
train: train/images
val: validation/images
nc: 1
names:
- knife



Train Model

In [16]:
!yolo detect train data=data.yaml model=yolo11s.pt epochs=60 imgsz=640 

Ultralytics 8.3.169  Python-3.12.7 torch-2.7.1+cpu CPU (Intel Core(TM) i5-10210U 1.60GHz)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=60, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretrained=True, profi


  0%|          | 0.00/18.4M [00:00<?, ?B/s]
  1%|          | 128k/18.4M [00:00<00:51, 370kB/s]
  1%|▏         | 256k/18.4M [00:00<00:43, 434kB/s]
  2%|▏         | 384k/18.4M [00:00<00:35, 534kB/s]
  3%|▎         | 512k/18.4M [00:01<00:32, 570kB/s]
  3%|▎         | 640k/18.4M [00:01<00:28, 645kB/s]
  4%|▍         | 768k/18.4M [00:01<00:25, 721kB/s]
  5%|▍         | 896k/18.4M [00:01<00:24, 745kB/s]
  5%|▌         | 1.00M/18.4M [00:01<00:25, 714kB/s]
  6%|▌         | 1.12M/18.4M [00:01<00:24, 751kB/s]
  7%|▋         | 1.25M/18.4M [00:02<00:26, 674kB/s]
  7%|▋         | 1.38M/18.4M [00:02<00:24, 734kB/s]
  8%|▊         | 1.50M/18.4M [00:02<00:21, 833kB/s]
  9%|▉         | 1.62M/18.4M [00:02<00:25, 679kB/s]
 10%|▉         | 1.75M/18.4M [00:02<00:23, 736kB/s]
 12%|█▏        | 2.12M/18.4M [00:02<00:15, 1.09MB/s]
 12%|█▏        | 2.25M/18.4M [00:03<00:19, 865kB/s] 
 13%|█▎        | 2.38M/18.4M [00:03<00:27, 610kB/s]
 14%|█▎        | 2.50M/18.4M [00:03<00:25, 648kB/s]
 14%|█▍        | 2.62M/1

test

In [17]:
!yolo detect predict model=yolo11s.pt source=data/validation/images save=True

Ultralytics 8.3.169  Python-3.12.7 torch-2.7.1+cpu CPU (Intel Core(TM) i5-10210U 1.60GHz)
YOLO11s summary (fused): 100 layers, 9,443,760 parameters, 0 gradients, 21.5 GFLOPs

image 1/5 d:\Yolo-intern-project\knife-detector-\backend\Research_notebooks\data\validation\images\113f5f00-Knife9.jpg: 640x640 1 knife, 358.5ms
image 2/5 d:\Yolo-intern-project\knife-detector-\backend\Research_notebooks\data\validation\images\236f9959-Knife15.jpg: 640x640 1 bench, 1 knife, 236.6ms
image 3/5 d:\Yolo-intern-project\knife-detector-\backend\Research_notebooks\data\validation\images\54b176de-Knife12.png: 640x640 1 knife, 234.7ms
image 4/5 d:\Yolo-intern-project\knife-detector-\backend\Research_notebooks\data\validation\images\682f3266-Knife4.png: 640x640 1 knife, 237.7ms
image 5/5 d:\Yolo-intern-project\knife-detector-\backend\Research_notebooks\data\validation\images\cf9bef69-Knife6.png: 640x640 1 knife, 230.1ms
Speed: 9.2ms preprocess, 259.5ms inference, 5.3ms postprocess per image at shape (1, 3, 6

In [ ]:
import os

# Get current working directory
current_dir = os.getcwd()
print("Current Directory:", current_dir)

# Navigate one directory up
parent_dir = os.path.dirname(current_dir)
os.chdir(parent_dir)

print("Moved to Parent Directory:", os.getcwd())


Current Directory: d:\Yolo-intern-project\knife-detector-\backend\Research_notebooks
Moved to Parent Directory: d:\Yolo-intern-project\knife-detector-\backend


In [22]:
import glob
from IPython.display import Image, display
from pathlib import Path

# Adjust the base path to where YOLO saves predictions
PREDICT_DIR = Path("runs/detect/predict")

# Display up to 10 predicted images
for image_path in glob.glob(str(PREDICT_DIR / "*.jpg"))[:10]:
    display(Image(filename=image_path, height=400))
    print('\n')
